# **<p style="text-align: center;">Aprendizaje por refuerzo - Ingeniería en Inteligencia Artificial</p>**
## **<p style="text-align: center;">Trabajo práctico Final - SAC</p>**

#### <p style="text-align: center;">Pettinari Fausto, Schuemer Ignacio, Torres Santiago </p>
#### <p style="text-align: center;">Profesores: Claudio Pose, Gabriel Torre, Nicolás Romero, Tomás Chimenti</p>

# **Bipedal Walker Enviroment**

In [17]:
import sys
sys.path.append('..')

In [18]:
import gymnasium as gym

In [19]:
env = gym.make("BipedalWalker-v3")

#### **Logging & Debugging (Random Agent)**

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from sac.random_agent import random_agent_loop

In [ ]:
seed = 42
# writer = SummaryWriter(f"../runs/BipedalWalker-v3/")
num_episodes = 100
# random_agent_loop(env, num_episodes, writer, seed)

## **SAC Agent**

In [15]:
import yaml
from sac.agent import SAC

In [16]:
with open('configs/bipedal_walker.yaml', 'r') as f:
    config = yaml.safe_load(f)

### **TRAIN**

In [11]:
SAC_agent = SAC(env=env, config=config)
SAC_agent.run_training_loop(num_episodes=400, tqdm_disable=False, print_rewards=False)

100%|██████████| 400/400 [1:13:39<00:00, 11.05s/it]

Agent saved to runs/BipedalWalker-v3/SAC/sac-bipedal-walker-2025_11_25-16_41_19/sac_agent.pth


{'total_episodes': 400,
 'best_avg_return': np.float32(222.04164),
 'final_avg_return': np.float32(222.04164)}

#### **Eval trained Agent**

In [ ]:
SAC_agent.eval_agent(num_episodes=1, render_mode="human")

### **Load trained agent & Eval**

In [8]:
# Load trained agent and run in environment
config['logger']['enabled'] = False  # Disable logging for loaded agent
SAC_agent_loaded = SAC(env=env, config=config)
SAC_agent_loaded.load_agent('../notebooks/runs/BipedalWalker-v3/SAC/sac-bipedal-walker-2025_11_25-16_41_19/sac_agent.pth')

In [9]:
SAC_agent_loaded.eval_agent(num_episodes=1, render_mode="human")

Creating new environment for evaluation with render_mode='human'


100%|██████████| 1/1 [00:52<00:00, 52.19s/it]


np.float32(318.96457)